In [1]:
import gym
import math
import numpy as np
import random
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
from gym import error, spaces, utils
from gym.utils import seeding
import time
import os

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Dropout, concatenate
from keras.layers import Concatenate,Conv2D,BatchNormalization,MaxPooling2D
from keras.optimizers import Adam
from keras import initializers

Using TensorFlow backend.
C:\Users\SER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\SER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [3]:
class BikeEnv(gym.Env):
    
    def __init__(self):
        p.connect(p.GUI)
        p.setRealTimeSimulation(1)
        p.resetDebugVisualizerCamera(cameraDistance=10, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(np.array([-1000]*10), np.array([1000]*10))
        self.timestep = 1./240.
        
    def step(self, action):

        if (action == 0):
            self.speed = self.speed + 1
        if (action == 1):
            self.speed = self.speed - 1 
        if (action == 2):
            self.speed = self.speed  
        if (action == 3):
            self.steer = self.steer - 1 
        if (action == 4):
            self.steer = self.steer + 1
        if (action == 5):
            self.steer = self.steer 
            
              
        self.applyAction([self.speed,self.steer])
        time.sleep(0.2)
        
        state = p.getLinkState(self.pid,0)[0]
        
        reward = np.sign(state[0] - self.origin[0])*1
        if state[2] <= 0.5 or  state[2] >= 2 or abs(self.speed)>2 or abs(self.steer)>4:
            #reward = -100
            done = True
            
        else :
            #reward = math.sqrt((self.origin[0]-state[0])**2+(self.origin[1]-state[1])**2)
            #reward = state[0] - self.origin[0]
            #reward = 1
            done = False
        self.origin = state 
        
        velocity = p.getBaseVelocity(self.pid)
        observation = list(self.getObservation()) + list(velocity[0])+list(velocity[1])
        
        info = {'x':'','y':'','z':''}
        #print("Step: ",self.stp)
        #xx = time.time()
        #print("Time: ",xx-self.tttt)
        #self.tttt = xx
        #print("Action: ",action)
        #print("Reward: ",reward)
        #self.stp +=1
        return observation, reward, done, info
            
    def applyAction(self, motorCommands):
        targetVelocity = motorCommands[0] * self.speedMultiplier
        #print("targetVelocity")
        #print(targetVelocity)
        steeringAngle = motorCommands[1] * self.steeringMultiplier
        #print("steeringAngle")
        #print(steeringAngle)


        for motor in self.motorizedwheels:
            p.setJointMotorControl2(self.pid,
                                    motor,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=targetVelocity,
                                    force=self.maxForce)
        for steer in self.steeringLinks:
            p.setJointMotorControl2(self.pid,
                                    steer,
                                    p.POSITION_CONTROL,
                                    targetPosition=steeringAngle)

    def reset(self):
        
        #print("===========Reset=====")
        self.stp=0
        self.tttt= time.time()
        p.resetSimulation()

        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])
        self.origin = p.getLinkState(self.pid,0)[0]
        p.setGravity(0,0,-10)
        for wheel in range(p.getNumJoints(self.pid)):
            p.setJointMotorControl2(self.pid,
                                    wheel,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=0,
                                    force=0)

        self.steeringLinks = [0]
        self.maxForce = 20
        self.nMotors = 2
        self.motorizedwheels = [1, 2]
        self.speedMultiplier = 10.
        self.steeringMultiplier = 0.5
        
        self.speed = 0 
        self.steer = 0

        velocity = p.getBaseVelocity(self.pid)
        observation = list(self.getObservation()) + list(velocity[0])+list(velocity[1])
        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        return observation
        
    def getObservationDimension(self):
        return len(self.getObservation())
    
    def getObservation(self):
        observation = []
        pos, orn = p.getBasePositionAndOrientation(self.pid)

        #observation.extend(list(pos))
        observation.extend(list(orn))
        return observation
        
    def render(self, mode='human'):
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0.7,0,0.05],
                                                            distance=.7,
                                                            yaw=90,
                                                            pitch=-70,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, _) = p.getCameraImage(width=960,
                                              height=720,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(rgb_array, (720,960, 4))

        rgb_array = rgb_array[:, :, :3]
        return rgb_array

    def close(self):
        p.disconnect()

In [4]:
class iDQNAgent:
    def __init__(self, state_space , action_space, episodes=500 , memory_size = 50000):
        self.action_space = action_space
        self.memory = []
        self.memory_size = memory_size
        self.gamma = 0.9
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = self.epsilon_min / self.epsilon
        self.epsilon_decay = self.epsilon_decay ** (1. / float(episodes))
        
        n_inputs = state_space.shape[0]
        n_outputs = action_space.n
        
        self.q_model = self.build_model(n_inputs , n_outputs)
        self.q_model.compile(loss='mse' , optimizer=Adam())
        
        self.target_q_model = self.build_model(n_inputs , n_outputs)
        
        self.update_weights()
        self.replay_counter = 0
        
        ####################################################
    def build_model(self, n_inputs , n_outputs):
        inputs = Input(shape = (n_inputs,) , name='state')
        x = Dense(256 , activation='relu')(inputs)
        x = Dense(256 , activation='relu')(x)
        x = Dense(256 , activation='relu')(x)
        x = Dense(n_outputs , activation='linear' , name = 'action')(x)
        model = Model(inputs , x)
        model.summary()
        return model
    
        #####################################################
    def act(self , state):
        if np.random.rand() < self.epsilon:
            return self.action_space.sample()
        q_values = self.q_model.predict(state)
        action = np.argmax(q_values[0])
        return action
    
        ######################################################
    def remember(self, state, action, reward , next_state , done):
        item = (state , action , reward , next_state ,done)
        if len(self.memory) > self.memory_size :
            self.memory.pop(0)
        self.memory.append(item)
        
        #######################################################
    def get_target_q_value(self, next_state, reward , double):
        if double :
            action = np.argmax(self.q_model.predict(next_state[0]))
            q_value = self.target_q_model.predict(next_state)[0][action]
        else :
            q_value = np.amax(self.target_q_model.predict(next_state)[0])
        
        q_value *= self.gamma
        q_value += reward
        return q_value
    
        ########################################################
    def replay(self , batch_size):
        sars_batch = random.sample(self.memory , batch_size)
       
        state_batch , q_value_batch = [] , []
        for state, action, reward, next_state, done in sars_batch:
            q_values = self.q_model.predict(state)
            
            q_value = self.get_target_q_value(next_state, reward, False)
            # ??????????
            q_values[0][action] = reward if done else q_value
            
            state_batch.append(state[0])
            q_value_batch.append(q_values[0])
            
        hist = self.q_model.fit(np.array(state_batch) , 
                             np.array(q_value_batch),
                             batch_size = batch_size,
                            epochs = 1,
                            verbose = 0)

        self.update_epsilon()
            
        if self.replay_counter % 10 == 0:
            self.update_weights()
                
        self.replay_counter += 1
        
        return hist.history['loss'][0]
       
       ######################################################

       ######################################################
    def update_epsilon(self):
            if self.epsilon > self.epsilon_min :
                self.epsilon *= self.epsilon_decay
        ########################################################
    def update_weights(self):
            self.target_q_model.set_weights(self.q_model.get_weights())
            

In [5]:
env = BikeEnv()
np.random.seed(123)
env.seed(123)

C:\Users\SER\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
state_size = env.observation_space.shape[0]

In [7]:
episode_count = 5000
batch_size = 256

In [8]:
agent = iDQNAgent(env.observation_space ,env.action_space, episode_count)




Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
state (InputLayer)           (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2816      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
action (Dense)               (None, 6)                 1542      
Total params: 135,942
Trainable params: 135,942
Non-trainable params: 0
_________________________________________________________________

Model: "model_2"
_________________________________________________________________
Layer (type)                 Out

In [9]:
all_rewards = []
all_losses =  []
loss = 0
for episode in range(episode_count):
    state = env.reset()
    state = np.reshape(state , [1,state_size])
    done = False
    total_reward = 0

    while not done :
        action = agent.act(state)
        next_state , reward , done , _ = env.step(action)
        next_state = np.reshape(next_state , [1,state_size])
        agent.remember(state , action , reward , next_state , done)
        state = next_state
        total_reward += reward
    
    if len(agent.memory) >= batch_size:
        loss = agent.replay(batch_size)
        all_losses.append(loss)
        
    print("episode {} reward {} loss {}".format(episode , total_reward, loss))
    all_rewards.append(total_reward)

episode 0 reward -8.0 loss 0
episode 1 reward 3.0 loss 0
episode 2 reward -4.0 loss 0
episode 3 reward -6.0 loss 0
episode 4 reward -7.0 loss 0
episode 5 reward -1.0 loss 0
episode 6 reward -1.0 loss 0
episode 7 reward -5.0 loss 0
episode 8 reward -9.0 loss 0
episode 9 reward -8.0 loss 0
episode 10 reward -9.0 loss 0
episode 11 reward -1.0 loss 0
episode 12 reward -5.0 loss 0
episode 13 reward -5.0 loss 0
episode 14 reward 3.0 loss 0
episode 15 reward -4.0 loss 0
episode 16 reward -5.0 loss 0
episode 17 reward -3.0 loss 0
episode 18 reward 5.0 loss 0
episode 19 reward 4.0 loss 0
episode 20 reward -3.0 loss 0
episode 21 reward -6.0 loss 0
episode 22 reward -7.0 loss 0
episode 23 reward -8.0 loss 0
episode 24 reward 3.0 loss 0
episode 25 reward -5.0 loss 0
episode 26 reward -4.0 loss 0
episode 27 reward 3.0 loss 0
episode 28 reward -8.0 loss 0
episode 29 reward -5.0 loss 0
episode 30 reward -7.0 loss 0
episode 31 reward 4.0 loss 0
episode 32 reward -5.0 loss 0
episode 33 reward -1.0 loss

KeyboardInterrupt: 

In [ ]:
plt.plot(range(episode_count),all_rewards)
plt.show()

In [ ]:
plt.plot(range(len(all_losses)),all_losses)
plt.show()

In [ ]:
agent.q_model.save_weights("d:\\iDQM-bike")